# Importing Dependancies

In [1]:
import numpy as np
import pandas as pd
import warnings
import copy
import random
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

# Read in Dataset

In [2]:
# read data into dataframe
df = pd.read_csv("./OnlineNewsPopularity.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39644 entries, 0 to 39643
Data columns (total 61 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   url                             39644 non-null  object 
 1    timedelta                      39644 non-null  float64
 2    n_tokens_title                 39644 non-null  float64
 3    n_tokens_content               39644 non-null  float64
 4    n_unique_tokens                39644 non-null  float64
 5    n_non_stop_words               39644 non-null  float64
 6    n_non_stop_unique_tokens       39644 non-null  float64
 7    num_hrefs                      39644 non-null  float64
 8    num_self_hrefs                 39644 non-null  float64
 9    num_imgs                       39644 non-null  float64
 10   num_videos                     39644 non-null  float64
 11   average_token_length           39644 non-null  float64
 12   num_keywords                   

# Preprocessing

### Split into data and target labels

In [3]:
y = df.iloc[:, 60]
x = df.iloc[:, 0:60]

### Independant Variables

In [4]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39644 entries, 0 to 39643
Data columns (total 60 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   url                             39644 non-null  object 
 1    timedelta                      39644 non-null  float64
 2    n_tokens_title                 39644 non-null  float64
 3    n_tokens_content               39644 non-null  float64
 4    n_unique_tokens                39644 non-null  float64
 5    n_non_stop_words               39644 non-null  float64
 6    n_non_stop_unique_tokens       39644 non-null  float64
 7    num_hrefs                      39644 non-null  float64
 8    num_self_hrefs                 39644 non-null  float64
 9    num_imgs                       39644 non-null  float64
 10   num_videos                     39644 non-null  float64
 11   average_token_length           39644 non-null  float64
 12   num_keywords                   

### Dependant Variables

In [5]:
# y.info()

### Drop Unused Features

In [6]:
# drop unused features
vals = [0, 1, 4, 5, 6]
for i in range(13, 39):
    vals.append(i)

x = x.drop(x.columns[vals], axis = 1)
x.head()

,n_tokens_title,n_tokens_content,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,LDA_00,LDA_01,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,12.0,219.0,4.0,2.0,1.0,0.0,4.680365,5.0,0.500331,0.378279,...,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500
1,9.0,255.0,3.0,1.0,1.0,0.0,4.913725,4.0,0.799756,0.050047,...,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000
2,9.0,211.0,3.0,1.0,1.0,0.0,4.393365,6.0,0.217792,0.033334,...,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000
3,9.0,531.0,9.0,0.0,1.0,0.0,4.404896,7.0,0.028573,0.419300,...,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000
4,13.0,1072.0,19.0,19.0,20.0,0.0,4.682836,7.0,0.028633,0.028794,...,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364


### Shuffle Data

In [7]:
np.random.seed(42)

In [8]:
# convert to numpy objects
x = x.to_numpy()
y = y.to_numpy()

In [9]:
# shuffle
arr = np.arange(len(df))
np.random.shuffle(arr)
x = x[arr]
y = y[arr]

### Splitting Data

In [10]:
x_train = x[:1000].copy()
y_train = y[:1000].copy()
x_val = x[1000:2000].copy()
y_val = y[1000:2000].copy()
x_test = x[2000:3000].copy()
y_test = y[2000:3000].copy()

### Standardize Data

In [11]:
# first save the training mean and std
training_mean = np.mean(x_train, axis=0)
training_std = np.std(x_train, axis=0)

# first standardize the training data
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
# standardize the test and validation data using training mean and std
x_test = (x_test - training_mean) / training_std
x_val = (x_val - training_mean) / training_std

### Clamping Data
I'll clamp the shares attribute to be at most 6000.

In [12]:
for i in range(1000):
    if y_train[i] > 6000:
        y_train[i] = 6000
    if y_test[i] > 6000:
        y_test[i] = 6000
    if y_val[i] > 6000:
        y_val[i] = 6000

### Map target data into two distinct classes
+1 is shares >= 2000 and -1 is shares < 2000

In [13]:
for i in range(1000):
    if y_train[i] >= 2000:
        y_train[i] = 1
    else:
        y_train[i] = -1
        
    if y_test[i] >= 2000:
        y_test[i] = 1
    else:
        y_test[i] = -1
        
    if y_val[i] >= 2000:
        y_val[i] = 1
    else:
        y_val[i] = -1

# Computing the Kernel Matrices
I will compute the linear and gaussian kernel and use each one to implement the Dual SVM algorithm  

### Linear Kernel

In [14]:
D = x_train.copy()

In [15]:
linear_kernel = np.dot(D, D.T)

In [16]:
linear_kernel

array([[23.56861317, -1.78768508, -9.04492404, ...,  8.87583137,
         6.88528597,  3.20230281],
       [-1.78768508, 12.57868004,  1.10172574, ..., -6.69551931,
        -2.99318777, -0.90015711],
       [-9.04492404,  1.10172574, 58.1960898 , ..., -8.16555486,
         3.26771302, -3.40711312],
       ...,
       [ 8.87583137, -6.69551931, -8.16555486, ..., 62.61216466,
        -1.25606695,  5.56410057],
       [ 6.88528597, -2.99318777,  3.26771302, ..., -1.25606695,
        17.72336005, -2.0619736 ],
       [ 3.20230281, -0.90015711, -3.40711312, ...,  5.56410057,
        -2.0619736 , 23.86693631]])

### Gaussian Kernel

Get the vector of squared norms

In [17]:
S = np.linalg.norm(D, axis = 1)
S = S * S

Set variance

In [18]:
variance = 10000

Compute Gaussian Kernel

In [19]:
gaussian_kernel = np.exp( ((2 * np.matmul(D, D.T)) - S - S[:,None]) / (2*variance) )
gaussian_kernel

array([[1.        , 0.99801584, 0.99501972, ..., 0.99658439, 0.99862488,
        0.99795056],
       [0.99801584, 1.        , 0.9965773 , ..., 0.9955807 , 0.99818722,
        0.99808953],
       [0.99501972, 0.9965773 , 1.        , ..., 0.99316649, 0.99653681,
        0.995566  ],
       ...,
       [0.99658439, 0.9955807 , 0.99316649, ..., 1.        , 0.99586618,
        0.99623954],
       [0.99862488, 0.99818722, 0.99653681, ..., 0.99586618, 1.        ,
        0.9977169 ],
       [0.99795056, 0.99808953, 0.995566  , ..., 0.99623954, 0.9977169 ,
        1.        ]])

In [47]:
def dual_SVM(D, kernel, C, epsilon):
    # augmented kernel matrix
    kernel_augmented = kernel + 1
    
    # make vector of step sizes
    eta = []
    for i in range(len(kernel_augmented)):
        val = 1 / kernel_augmented[i][i]
        eta.append(val)
    eta = np.array(eta)
    # initialize alpha vector
    t = 0
    alpha = np.random.uniform(0, 1, len(kernel_augmented))
    old_alpha = alpha.copy()
    while(True):
        # iterate thru n in random order
        r = list(range(len(kernel_augmented)))
        random.shuffle(r)
        # update the kth component in alpha
        for k in r:
            # first calculate the gradient
            # do the summation
            summation = 0
            for i in range(len(kernel)):
                summation += (alpha[i] * y_train[i] * kernel_augmented[i][k])
            # print(summation)
            # multiply the gradient by the step size
            t = eta[k] * (1 - (y_train[k] * summation))
            
            # update alpha in place
            alpha[k] = alpha[k] + t
            # finally clamp alpha between 0 and C
            if alpha[k] < 0:
                alpha[k] = 0
            if alpha[k] > C:
                alpha[k] = C
        
        t += 1
        # print(np.linalg.norm(alpha - old_alpha))
        if np.linalg.norm(alpha - old_alpha) < epsilon:
            break
        
        old_alpha = alpha.copy()
    
    
    return alpha

In [50]:
a = dual_SVM(D, linear_kernel, 1e-4, 1e-3)

28.740474759980916
-144.52523021048083
-41.91005655761233
-194.91014383134006
145.2279296919851
-64.95218804483186
-229.82657027000323
41.51252337075599
-128.985684492533
63.003989907380536
207.4052065022718
-347.3434734389483
-230.50233215090722
-206.64609097994884
164.47039211122217
-598.2933367597882
-243.11512099025205
124.72048034841521
-85.24334012177282
-73.96893322978246
-13.5223379338727
-59.71149805983655
-180.9924404269604
-20.499637952505324
-435.01539270337565
-424.95348001234174
-10.871918880925787
-81.15606357151385
-119.79412811495172
-28.95936686467598
-58.007715068366664
-82.31405301201364
-352.07980782798325
0.9491981560186014
-463.9213378411739
27.568007752114774
-203.8849339457257
61.72047252566823
182.59439586370956
-108.13878794006723
-143.28348197880973
0.9758605935379636
-16.11527039025641
-157.91472576306535
-333.6984364964986
-135.0531503394595
-12.47825603860884
-256.5862705420878
19.088691077827647
-110.62847912972774
-72.71700519143276
36.168212855974616
-

-1.3672435345246159
-2.8789223889376183
-4.953111514457862
4.3078991051743625
-4.776432896508411
-1.6485062168185203
4.38103435409351
4.878336393141775
5.642877884167442
-3.465748746887981
0.5901927333031298
-2.8448392683923216
3.931959479881994
-0.8716604074350915
5.378196918034295
-0.9201334515622599
2.1586088581040785
3.926306610196533
-0.30868573564304774
0.8844598859844484
9.578418276870066
3.1245590603674858
-2.0932072469408505
-0.7701007808568432
-3.341086549245314
-3.8409508485181414
1.61412317755086
-2.3040158066561287
-0.20320185993978696
5.489910008819428
-1.3645012558807168
-3.566588148056884
-0.32944904632761274
-3.766033711562917
-0.29322972437637534
2.1725137031363517
9.153392880943676
-2.5500118282153803
-2.4288312106154315
-3.737173205664361
-0.009318569590054393
3.689765077600942
-2.975721350813826
-1.3911937038226205
-0.5999515475187134
-0.19060194603856517
4.58572383819602
-3.7086507794109327
3.32380834992859
1.3056600156733276
4.975642089232273
-3.2655576652794505


-0.2102844456808341
-0.8897638809409434
-1.2513058076580919
-0.8819486485182031
0.4701431584936646
-0.018096948682892355
-0.42784450213999836
1.4758403413776342
-1.2056706982672496
1.9871267403443507
1.4998687543660036
-0.4446460874870357
0.4692691538905971
1.2875728928506511
-1.5645939770399728
1.8189716526967232
-0.25418551501235687
1.6091682352744692
0.4386980786795577
-0.6880660567232844
-0.8931691465998434
-0.13142453353505734
-0.1296037669945248
-3.594262993497227
-1.6537163171553468
-1.5634872491235243
-0.8759246651096262
-1.5453406588212961
1.488480859507927
-2.037661059742278
-2.0464145933071176
-1.9030694849030727
-0.2601275543130583
-1.5775582131109083
-0.5590622215987503
1.8079579430775365
-0.9840842364186329
-0.4089989180764162
-0.44230332423703556
-1.71078973524424
-1.2683405504701482
2.6355412246564645
-1.7887528674427062
-7.56118913966075
-0.8185159274304535
2.608068431429678
-0.13359847607325914
1.0897441629159212
-1.819551803802887
-1.344736092317517
0.172230302192854

1.3502128708967804
0.8325575068831277
1.8612257554483216
0.07629320862071376
-0.9277794729924134
-1.6255665476285763
-1.747906199951583
0.3415498328679525
0.013327826835622725
-2.9996893272938077
0.3159522353909735
-0.5520148252812143
1.9899661851053982
0.7741297019344202
2.6195328230698167
-2.3404777497963156
0.2155183512869927
0.56044886883763
0.6057816848761024
1.8235899536947733
-0.6818616633767922
-0.8736938575054813
1.5072902797176324
-0.43816097678372223
-1.9365738874765297
-1.3038952795668832
1.6084508543198732
0.9496507316902383
-1.9794393188543276
0.5139610912665491
0.4008937499881573
0.11836188047439489
-1.0585706741278753
2.8629446457980627
0.019000793815360684
-0.5806158118713394
-1.6527538149491865
-1.3444686308687706
-1.6391693011139998
-2.9987824954001856
-1.0936992886036592
-1.101956904718964
-0.8810177749797228
-1.509512305677662
0.1265888720366748
3.7003880855104283
-0.758611583005732
-0.3835501868691997
-2.0959840978959807
-1.2284782119181084
-2.3012989167020934
-3.

-1.3639138137470668
-2.270096834601368
-1.6909691942682865
-0.08439440607204751
1.0044842735910513
0.007830106844074258
-0.47940542976554434
0.5786873926530711
-0.602169403835005
0.40934028536573663
1.6532370537626164
1.9942055112581316
-1.1821145144299514
-0.6841404775351794
-0.9093402578628089
-0.6870514470490653
-0.4691069303647739
-0.9092623406188829
-0.9372449496839355
-0.5635765870485328
0.2994115992726567
0.8617236076356498
2.3028840744794152
1.840722674231018
0.8215998360195698
-1.5105380251119371
0.9877866709773238
0.6142454213038118
-0.6620833952542027
-6.7639436307433085
-0.5298331696142546
-0.5506581752222979
1.13724280718966
-1.2722652901179117
-0.2503948166398655
0.30935485708442556
-1.6563056645139114
-1.4057793813857151
0.44655897190821525
0.18206465539975683
-0.7711180303609515
0.7976768233340532
1.61404969012996
-0.475095366136044
-1.003420275513611
-0.11110178528506709
0.788784646278098
0.6392712876080553
-0.3207512133397361
-0.5787062143517567
-0.8192913170495456
0.

...

-0.14744421838396782
-0.879911007499055
-0.006093109572373523
-1.1356888862746777
-0.658393946453153
-0.16725255312692333
-0.0414071750176741
0.06946471308157132
0.32071164233308525
-1.310819566584544
-0.4248393349789752
-1.172974839864214
-0.9516142088195423
-0.9367164850125278
0.6404486576048318
0.7994021229121007
1.2881160916513281
-0.3040390349796036
-2.1109872129769363
-0.7897634279624638
-1.6551882474671031
-0.15041233676961907
-0.31495614576336983
0.31713242923207896
-1.3732007424364165
0.011425437648629355
0.15593428069028503
0.40277324060126674
-2.6720276096343802
3.557596762713542
-0.3988298866697195
2.514605837198321
-1.572676407522362
-0.6645221702404704
1.159698387647417
0.7534295881270826
0.6860378548733892
0.27426763104364094
-0.05368342566280035
-3.0186892694407885
0.3064012380409866
-0.13372699324541903
0.0012403030398395742
-0.7364179119673455
-0.5269914233889683
0.26249000305146986
0.06340856533531736
-0.7127562253279329
-1.8406393882523842
-1.3162600007508407
-0.294

1.2014137214927836
0.7762689583397981
-0.9945161820333166
-2.003987231151654
2.4857278639796543
-0.5237060982035051
0.8671367645829566
-1.5858912558131024
-1.0520394275520397
-0.22225489879270044
0.3746796351397115
2.02013200061584
0.39952244131857256
-1.1108273748217656
0.2803132646844273
-0.8644318119068671
0.7955314184933071
-0.7340841426929869
0.09010668818623335
0.30668491651693497
-1.8216676724016263
-0.7774953020748041
-2.20678420730207
-0.10473092730535782
-0.2399942355107365
-0.8894143556644472
-0.29033041579728414
-0.6674045870328773
-0.4281943724997963
-0.9119712561643757
1.5416850651001563
0.5204049244281497
-2.1449534873063074
-1.260948050514723
0.8671269465311529
-0.7899311265813345
0.5911425716312608
-1.2450549890018325
-0.4040636824474926
-3.722435618311194
0.8117783251070949
-0.2574685493989044
-1.435316794717372
0.19151462734259894
-1.2574350029441965
0.5740368804266587
1.1598643595520237
0.74776347622723
1.085112468880367
0.006581974198522245
0.0029335632913123355
-0

-0.4546942255854032
1.6384989885970485
-1.9288967291924104
0.22500382392594576
-0.9011573776278092
1.1323393469704819
1.374348809325471
0.60162790132342
0.8887749724821159
-1.0919001320723003
-0.863875917510021
-0.8756860742677741
0.5414363638115232
-1.314929028814629
-0.7670246091191165
-1.9945005633421564
-0.16078907672151077
-1.483520072858442
-0.16550712267814482
0.9506764238773919
1.012690713494678
0.12979666202739312
-1.431012992168026
-0.8495820437071531
1.1771103736173858
-0.43803564429087344
-1.3175531273433858
-0.08299402555808044
0.4977303105681874
2.214731827409488
-0.17834928829783356
0.11223115484227808
-0.8171744221250323
-1.3360730151921163
-1.135425176636155
1.288240815521085
-1.160665607873363
-0.09589514646563319
-1.076618581067153
-0.8976269546341485
0.2193935144680138
-0.2158900678965101
-0.7232328641209098
-0.4649975356527236
0.5221436116591536
-0.6065378169735212
0.10543630103765222
-0.5477729575315966
-1.0401610468899212
-1.0671915963753973
-0.6146976053211097
-

0.31355341054764685
-0.6045687131758217
0.48895342893575355
-1.3068956725975323
-2.0888061206242217
-1.5403941525640195
0.6503575955022962
-1.2134685283339781
0.40451856868647207
-1.9065569066417136
-0.21119638353276637
-1.1836444703263256
-1.5573734757446673

KeyboardInterrupt: 

In [54]:
K_new = np.dot(x_train, x_test.T)

In [60]:
np.sum(K_new[:, 200]) * y_train[200] * a[200]

2.899014361901209e-16

In [56]:
a.shape

(1000,)

In [58]:
y_train

array([ 1, -1,  1, -1, -1,  1, -1,  1, -1, -1,  1, -1,  1,  1, -1, -1,  1,
       -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1,  1, -1,
        1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1,  1,  1,  1,
       -1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1,
        1, -1,  1, -1,  1,  1,  1, -1,  1, -1,  1,  1, -1,  1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1, -1, -1,
       -1,  1,  1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1, -1,  1,
       -1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1, -1,  1,  1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1,
       -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1,
       -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1, -1, -1, -1, -1,  1, -1,
       -1, -1,  1, -1, -1